<a href="https://colab.research.google.com/github/nikolito/ogiNikki_flair/blob/main/flair_nikki_train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Named Entity Recognition (NER) for Ogihan Nikki Mokuroku titles**

  This script is based on Flair modeling scripts (https://colab.research.google.com/drive/1Mi8GN_x_jQUgOCDjkkjtlN_OXXO3dCF7) created by Hironsan. We try to extract NER class such as persons, places, roles etc. from titles on Ogihan Nikki Database (OND). Please refer to https://crch.dl.saga-u.ac.jp/nikki/ for further information.
  OND has a corpus of ca. 74000 sentences that freely available under the Creative Commons CC BY-NC-SA 4.0. We have prepared OND corpus approximately 40000 sentences with NER annotation labels. Thanks to Google Colaboratory, we can create models with up to 30000 sentences here. (Please reduce the mini batch size from 32 to 16 preventing OOM in case of 30000 sentences. )
  Once creating a model, you can deploy NER with titles on OND. We recommend that you try "uid" 40000 or later for real prediction. 

In [ ]:
!nvidia-smi

Tue Oct 27 17:06:40 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.23.05    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P8     9W /  70W |      0MiB / 15079MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!pip install flair

     |████████████████████████████████| 337kB 8.4MB/s 
     |████████████████████████████████| 1.1MB 28.0MB/s 
     |████████████████████████████████| 1.3MB 57.3MB/s 
     |████████████████████████████████| 71kB 11.5MB/s 
     |████████████████████████████████| 798kB 52.6MB/s 
     |████████████████████████████████| 276kB 59.0MB/s 
     |████████████████████████████████| 983kB 50.9MB/s 
     |████████████████████████████████| 19.7MB 1.2MB/s 
     |████████████████████████████████| 2.9MB 59.8MB/s 
     |████████████████████████████████| 890kB 49.4MB/s 
  Created wheel for ftfy: filename=ftfy-5.8-cp36-none-any.whl size=45612 sha256=943e0416806713884d2ba09e64162bc897cdfde10b389b80aed047fead460a59
  Stored in directory: /root/.cache/pip/wheels/ba/c0/ef/f28c4da5ac84a4e06ac256ca9182fc34fa57fefffdbc68425b
  Created wheel for mpld3: filename=mpld3-0.3-cp36-none-any.whl size=116677 sha256=e8150150f9065d9c9690eb6e0754c76aa87bbb8ba19ae3a885be807bab6c1246
  Stored in directory: /root/.cache/pip/wh

In [ ]:
!wget https://crch.dl.saga-u.ac.jp/nikki/dataset/iob-10000-3.txt
iob_file = 'iob-10000-3.txt'

--2020-10-27 17:07:20--  https://crch.dl.saga-u.ac.jp/nikki/dataset/iob-10000-3.txt
Resolving crch.dl.saga-u.ac.jp (crch.dl.saga-u.ac.jp)... 153.127.22.73
Connecting to crch.dl.saga-u.ac.jp (crch.dl.saga-u.ac.jp)|153.127.22.73|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1687296 (1.6M) [text/plain]
Saving to: ‘iob-10000-3.txt.1’

iob-10000-3.txt.1   100%[===================>]   1.61M  1.82MB/s    in 0.9s    

2020-10-27 17:07:22 (1.82 MB/s) - ‘iob-10000-3.txt.1’ saved [1687296/1687296]



In [ ]:
from flair.datasets import ColumnCorpus
from flair.embeddings import StackedEmbeddings, FlairEmbeddings
from flair.data import Sentence
from flair.models import SequenceTagger
from flair.trainers import ModelTrainer

In [ ]:
columns = {0: 'text', 1: 'ner'}
data_folder = '.'
corpus = ColumnCorpus(data_folder, columns,
                      train_file=iob_file)
print(corpus)

2020-10-27 17:07:28,090 Reading data from .
2020-10-27 17:07:28,092 Train: iob-10000-3.txt
2020-10-27 17:07:28,096 Dev: None
2020-10-27 17:07:28,097 Test: None
Corpus: 8100 train + 900 dev + 1000 test sentences


In [ ]:
tag_type = 'ner'
tag_dictionary = corpus.make_tag_dictionary(tag_type=tag_type)
print(tag_dictionary.idx2item)

[b'<unk>', b'O', b'B-PLACE', b'I-PLACE', b'B-TERMS', b'I-TERMS', b'B-JINMEI', b'I-JINMEI', b'B-EVENT', b'I-EVENT', b'B-ROLE', b'I-ROLE', b'B-QUANTITY', b'I-QUANTITY', b'B-DATE', b'I-DATE', b'<START>', b'<STOP>']


In [ ]:
embedding_types = [
    FlairEmbeddings('ja-forward',with_whitespace = False),
    FlairEmbeddings('ja-backward',with_whitespace = False),
]
embeddings = StackedEmbeddings(embeddings=embedding_types)

2020-10-27 17:07:30,654 https://flair.informatik.hu-berlin.de/resources/embeddings/flair/japanese-forward.pt not found in cache, downloading to /tmp/tmpujxrs1hz


100%|██████████| 335383050/335383050 [00:12<00:00, 26097614.76B/s]

2020-10-27 17:07:43,822 copying /tmp/tmpujxrs1hz to cache at /root/.flair/embeddings/japanese-forward.pt


2020-10-27 17:07:44,547 removing temp file /tmp/tmpujxrs1hz
2020-10-27 17:08:00,444 https://flair.informatik.hu-berlin.de/resources/embeddings/flair/japanese-backward.pt not found in cache, downloading to /tmp/tmpj1esrkzh


100%|██████████| 335383032/335383032 [00:13<00:00, 24889879.68B/s]

2020-10-27 17:08:14,236 copying /tmp/tmpj1esrkzh to cache at /root/.flair/embeddings/japanese-backward.pt


2020-10-27 17:08:15,130 removing temp file /tmp/tmpj1esrkzh


In [ ]:
tagger = SequenceTagger(hidden_size=256,
                        embeddings=embeddings,
                        tag_dictionary=tag_dictionary,
                        tag_type=tag_type,
                        use_crf=True)

In [ ]:
trainer = ModelTrainer(tagger, corpus)
trainer.train('resources/taggers/example-ner',
              learning_rate=0.1,
              mini_batch_size=32,
              max_epochs=150)

2020-10-27 17:08:16,441 ----------------------------------------------------------------------------------------------------
2020-10-27 17:08:16,443 Model: "SequenceTagger(
  (embeddings): StackedEmbeddings(
    (list_embedding_0): FlairEmbeddings(
      (lm): LanguageModel(
        (drop): Dropout(p=0.3, inplace=False)
        (encoder): Embedding(15174, 100)
        (rnn): LSTM(100, 2048, num_layers=2, dropout=0.3)
        (decoder): Linear(in_features=2048, out_features=15174, bias=True)
      )
    )
    (list_embedding_1): FlairEmbeddings(
      (lm): LanguageModel(
        (drop): Dropout(p=0.3, inplace=False)
        (encoder): Embedding(15174, 100)
        (rnn): LSTM(100, 2048, num_layers=2, dropout=0.3)
        (decoder): Linear(in_features=2048, out_features=15174, bias=True)
      )
    )
  )
  (word_dropout): WordDropout(p=0.05)
  (locked_dropout): LockedDropout(p=0.5)
  (embedding2nn): Linear(in_features=4096, out_features=4096, bias=True)
  (rnn): LSTM(4096, 256, batch_f

{'dev_loss_history': [5.599512577056885,
  4.080484390258789,
  3.3933682441711426,
  3.063971757888794,
  2.8310115337371826,
  2.7357428073883057,
  2.4002184867858887,
  2.4102425575256348,
  2.4524059295654297,
  2.1729090213775635,
  2.260093927383423,
  2.155740976333618,
  2.0907511711120605,
  2.072341203689575,
  2.0636537075042725,
  2.0752480030059814,
  2.053154230117798,
  2.003392457962036,
  2.009709119796753,
  1.9953813552856445,
  2.0428969860076904,
  1.980286955833435,
  1.9705686569213867,
  1.9890930652618408,
  1.9748477935791016,
  2.018021583557129,
  2.0003604888916016,
  2.056325674057007,
  2.036849021911621,
  2.0409984588623047,
  1.9902448654174805,
  1.9912399053573608,
  1.946303367614746,
  2.0211832523345947,
  2.036879062652588,
  2.022401809692383,
  2.088564157485962,
  2.088327646255493,
  2.060406446456909,
  2.083144426345825,
  2.0058534145355225,
  2.0916855335235596,
  2.104879140853882,
  2.0932247638702393,
  2.144139289855957,
  2.11207509

In [ ]:
model = SequenceTagger.load('resources/taggers/example-ner/final-model.pt')

2020-10-27 18:00:24,011 loading file resources/taggers/example-ner/final-model.pt


In [ ]:
strings = "徳見傳助相達之事"
str_space = " ".join(strings)
sentence = Sentence(str_space)
model.predict(sentence)
print(sentence.to_tagged_string())

徳 <B-JINMEI> 見 <I-JINMEI> 傳 <I-JINMEI> 助 <I-JINMEI> 相 <B-EVENT> 達 <I-EVENT> 之 <B-TERMS> 事 <I-TERMS>


In [ ]:
!zip -r /content/resources/download.zip /content/resources/taggers/example-ner

  adding: content/resources/taggers/example-ner/ (stored 0%)
  adding: content/resources/taggers/example-ner/final-model.pt (deflated 8%)
  adding: content/resources/taggers/example-ner/best-model.pt (deflated 8%)
  adding: content/resources/taggers/example-ner/loss.tsv (deflated 58%)
  adding: content/resources/taggers/example-ner/test.tsv (deflated 88%)
  adding: content/resources/taggers/example-ner/training.log (deflated 83%)
  adding: content/resources/taggers/example-ner/weights.txt (stored 0%)


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!ls -lh /content/resources/

total 1.4G
-rw-r--r-- 1 root root 1.4G Oct 27 18:01 download.zip
drwxr-xr-x 3 root root 4.0K Oct 27 17:08 taggers


In [ ]:
!cp resources/download.zip drive/My\ Drive/